# Tensorflow로 구현한 단일 뉴런 (Single Neuron)

## Gate Neuron 

In [3]:
import tensorflow as tf
import numpy as np
import random
import math

In [21]:
class GateNeuron:
    def __init__(self):
        self.x = tf.placeholder(tf.float32, [1, 2])
        self.z_target = tf.placeholder(tf.float32, [1])
        self.weight = tf.Variable(tf.zeros([2, 1]))  # [[0.]  [0.]] 
        self.bias = tf.Variable(tf.zeros([1]))    # [[0.]]

    def u(self):
        '''Affine Sum'''
        u = tf.add(tf.matmul(self.x, self.weight), self.bias)
        return u

    def f(self, x):  
        '''Activation Function'''
        f = tf.nn.relu(x)
        return f

    def z(self):
        z = self.f(self.u())
        return z
    
    def f_derivative(self, x):
        delta = 0.00000001
        return (self.f(x + delta) - self.f(x)) / delta

    def d_E_over_d_w(self):
        error = self.z() - self.z_target
        return error * self.f_derivative(self.u()) * self.x

    def d_E_over_d_b(self):
        error = self.z() - self.z_target
        return error * self.f_derivative(self.u())
    
    def optimize(self, sess, alpha, x_input, z_target):
        op1 = tf.assign(self.weight, self.weight - alpha * tf.transpose(self.d_E_over_d_w()))
        op2 = tf.assign(self.bias, self.bias - tf.reshape(alpha * tf.transpose(self.d_E_over_d_b()), shape=(1,)))
        sess.run(op1, feed_dict = {self.x: x_input, self.z_target: z_target})
        sess.run(op2, feed_dict = {self.x: x_input, self.z_target: z_target})

    def squared_error(self, sess, x_input, z_target):
        s_error = 1.0 / 2.0 * tf.pow(self.z() - self.z_target, 2)
        s_error = sess.run(s_error, feed_dict = {self.x: x_input, self.z_target: z_target})
        return s_error

    def learning(self, sess, alpha, maxEpoch, data):
        for i in range(maxEpoch):
            for idx in range(data.numTrainData):
                x_input = data.training_input_value[idx]
                z_target = data.training_z_target[idx]
                self.optimize(sess, alpha, x_input, z_target)

            sum = 0.0
            for idx in range(data.numTrainData):
                x_input = data.training_input_value[idx]
                z_target = data.training_z_target[idx]
                sum = sum + self.squared_error(sess, x_input, z_target)
            weight_ = self.weight.eval()    
            weight = weight_[0][0], weight_[1][0]
            bias = self.bias.eval()[0]
            print("Epoch {0}: Error: {1}, w: {2}, {3}, b: {4}".format(i, sum / data.numTrainData, weight[0], weight[1], bias))
     
    def predict(self, sess, data, criterion):
        for idx in range(data.numTrainData):
            x_input = data.training_input_value[idx]
            classification = tf.greater_equal(self.z(), [criterion])
            result = sess.run(classification, feed_dict={self.x: x_input})
            if result[0]:
                r = 1
            else:
                r = 0    
            print("{0}, {1} - {2}".format(x_input[0][0], x_input[0][1], r))                

### 1. And Gate

In [22]:
class Data:
    def __init__(self):
        self.training_input_value = np.array([[[0.0, 0.0]], [[1.0, 0.0]], [[0.0, 1.0]], [[1.0, 1.0]]])
        self.training_z_target = np.array([[0.0], [0.0], [0.0], [1.0]])
        self.numTrainData = len(self.training_input_value)

if __name__ == '__main__':
    n = GateNeuron()
    d = Data()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())            
        n.learning(sess, 0.1, 20, d)
        n.predict(sess, d, 0.5)

Epoch 0: Error: [[ 0.07172403]], w: 0.10000000149011612, 0.10000000149011612, b: 0.11920928955078125
Epoch 1: Error: [[ 0.05533679]], w: 0.18762078881263733, 0.19159507751464844, b: 0.06036752089858055
Epoch 2: Error: [[ 0.06575251]], w: 0.15167297422885895, 0.1596134901046753, b: -0.004186214879155159
Epoch 3: Error: [[ 0.04755314]], w: 0.24333524703979492, 0.25287941098213196, b: -0.031557440757751465
Epoch 4: Error: [[ 0.05693943]], w: 0.21177789568901062, 0.22390156984329224, b: -0.08307219296693802
Epoch 5: Error: [[ 0.06420558]], w: 0.19259925186634064, 0.2041323482990265, b: -0.09964428842067719
Epoch 6: Error: [[ 0.06821631]], w: 0.1856735646724701, 0.19682493805885315, b: -0.1128169372677803
Epoch 7: Error: [[ 0.04245147]], w: 0.2923172116279602, 0.30301201343536377, b: -0.12328707426786423
Epoch 8: Error: [[ 0.05072908]], w: 0.2671297490596771, 0.27942484617233276, b: -0.1647936999797821
Epoch 9: Error: [[ 0.05441121]], w: 0.25950512290000916, 0.27140992879867554, b: -0.17926

### 2. Or Gate

In [23]:
class Data:
    def __init__(self):
        self.training_input_value = np.array([[[0.0, 0.0]], [[1.0, 0.0]], [[0.0, 1.0]], [[1.0, 1.0]]])
        self.training_z_target = np.array([[0.0], [1.0], [1.0], [1.0]])
        self.numTrainData = len(self.training_input_value)

if __name__ == '__main__':
    n = GateNeuron()
    d = Data()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())            
        n.learning(sess, 0.1, 20, d)
        n.predict(sess, d, 0.5)

Epoch 0: Error: [[ 0.18464352]], w: 0.10000000149011612, 0.06950980424880981, b: 0.19571706652641296
Epoch 1: Error: [[ 0.16136609]], w: 0.10000000149011612, 0.18334531784057617, b: 0.16655294597148895
Epoch 2: Error: [[ 0.13817059]], w: 0.21299034357070923, 0.18334531784057617, b: 0.14173462986946106
Epoch 3: Error: [[ 0.14705493]], w: 0.21299034357070923, 0.18334531784057617, b: 0.12061452120542526
Epoch 4: Error: [[ 0.15097043]], w: 0.21299034357070923, 0.18334531784057617, b: 0.11162804067134857
Epoch 5: Error: [[ 0.15466616]], w: 0.21299034357070923, 0.18334531784057617, b: 0.10331110656261444
Epoch 6: Error: [[ 0.15814817]], w: 0.21299034357070923, 0.18334531784057617, b: 0.09561382979154587
Epoch 7: Error: [[ 0.16142356]], w: 0.21299034357070923, 0.18334531784057617, b: 0.08849004656076431
Epoch 8: Error: [[ 0.16450012]], w: 0.21299034357070923, 0.18334531784057617, b: 0.08189702033996582
Epoch 9: Error: [[ 0.16738617]], w: 0.21299034357070923, 0.18334531784057617, b: 0.07579521

### 3. XOR Gate

In [24]:
class Data:
    def __init__(self):
        self.training_input_value = np.array([[[0.0, 0.0]], [[1.0, 0.0]], [[0.0, 1.0]], [[1.0, 1.0]]])
        self.training_z_target = np.array([[0.0], [1.0], [1.0], [0.0]])
        self.numTrainData = len(self.training_input_value)

if __name__ == '__main__':
    n = GateNeuron()
    d = Data()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())            
        n.learning(sess, 0.1, 20, d)
        n.predict(sess, d)

Epoch 0: Error: [[ 0.15095024]], w: 0.10000000149011612, 0.06950980424880981, b: 0.19571706652641296
Epoch 1: Error: [[ 0.14884065]], w: 0.10000000149011612, 0.18334531784057617, b: 0.16655294597148895
Epoch 2: Error: [[ 0.14768817]], w: 0.21299034357070923, 0.18334531784057617, b: 0.14173462986946106
Epoch 3: Error: [[ 0.15129247]], w: 0.21299034357070923, 0.18334531784057617, b: 0.12061452120542526
Epoch 4: Error: [[ 0.15296134]], w: 0.21299034357070923, 0.18334531784057617, b: 0.11162804067134857
Epoch 5: Error: [[ 0.15457785]], w: 0.21299034357070923, 0.18334531784057617, b: 0.10331110656261444
Epoch 6: Error: [[ 0.15613554]], w: 0.21299034357070923, 0.18334531784057617, b: 0.09561382979154587
Epoch 7: Error: [[ 0.15762998]], w: 0.21299034357070923, 0.18334531784057617, b: 0.08849004656076431
Epoch 8: Error: [[ 0.15905829]], w: 0.21299034357070923, 0.18334531784057617, b: 0.08189702033996582
Epoch 9: Error: [[ 0.1604189]], w: 0.21299034357070923, 0.18334531784057617, b: 0.075795218

TypeError: predict() missing 1 required positional argument: 'criterion'